In [8]:
from PIL import Image
import os

In [9]:
input_dir = "./data/raw/full/output_normalized"
output_dir = "./data/prepared"
os.makedirs(output_dir, exist_ok=True)

def pad_image(img, target_w, target_h):
    new_img = Image.new("RGB", (target_w, target_h), (255, 255, 255))
    new_img.paste(img, ((target_w - img.width) // 2, (target_h - img.height) // 2))
    return new_img

# --- อ่านไฟล์และ sort ---
files = sorted([f for f in os.listdir(input_dir) if f.endswith("_mangosteen_1.jpg")])

# Group images by index (i001, i002, ...)
groups = {}
for f in files:
    idx = f.split("_")[0]  # เช่น i001
    if idx not in groups:
        groups[idx] = {}
    if "upper" in f:
        groups[idx]["upper"] = f
    elif "under" in f:
        groups[idx]["under"] = f
    elif "side" in f:
        groups[idx]["side"] = f

# --- รวมภาพแต่ละกลุ่ม ---
for idx, imgs in sorted(groups.items()):
    if not all(k in imgs for k in ["upper", "under", "side"]):
        print(f"⚠️ Skip {idx}, missing images")
        continue

    upper = Image.open(os.path.join(input_dir, imgs["upper"]))
    under = Image.open(os.path.join(input_dir, imgs["under"]))
    side = Image.open(os.path.join(input_dir, imgs["side"]))
    side_flip = side.transpose(Image.Transpose.FLIP_LEFT_RIGHT)

    # Normalize ขนาด
    max_w = max(img.width for img in [upper, under, side, side_flip])
    max_h = max(img.height for img in [upper, under, side, side_flip])
    upper = pad_image(upper, max_w, max_h)
    under = pad_image(under, max_w, max_h)
    side = pad_image(side, max_w, max_h)
    side_flip = pad_image(side_flip, max_w, max_h)

    # รวมเป็น grid
    combined = Image.new("RGB", (max_w*2, max_h*2), (255, 255, 255))
    combined.paste(upper, (0, 0))
    combined.paste(under, (max_w, 0))
    combined.paste(side, (0, max_h))
    combined.paste(side_flip, (max_w, max_h))

    # Resize 224x224
    final_img = combined.resize((224, 224))
    output_path = os.path.join(output_dir, f"{idx}_mangosteen_grid.jpg")
    final_img.save(output_path)

    print(f"✅ Saved {output_path}")


✅ Saved ./data/prepared/i001_mangosteen_grid.jpg
✅ Saved ./data/prepared/i002_mangosteen_grid.jpg
✅ Saved ./data/prepared/i003_mangosteen_grid.jpg
✅ Saved ./data/prepared/i004_mangosteen_grid.jpg
✅ Saved ./data/prepared/i005_mangosteen_grid.jpg
✅ Saved ./data/prepared/i006_mangosteen_grid.jpg
✅ Saved ./data/prepared/i007_mangosteen_grid.jpg
✅ Saved ./data/prepared/i008_mangosteen_grid.jpg
✅ Saved ./data/prepared/i009_mangosteen_grid.jpg
✅ Saved ./data/prepared/i010_mangosteen_grid.jpg
✅ Saved ./data/prepared/i011_mangosteen_grid.jpg
✅ Saved ./data/prepared/i012_mangosteen_grid.jpg
✅ Saved ./data/prepared/i013_mangosteen_grid.jpg
✅ Saved ./data/prepared/i014_mangosteen_grid.jpg
✅ Saved ./data/prepared/i015_mangosteen_grid.jpg
✅ Saved ./data/prepared/i016_mangosteen_grid.jpg
✅ Saved ./data/prepared/i017_mangosteen_grid.jpg
✅ Saved ./data/prepared/i018_mangosteen_grid.jpg
✅ Saved ./data/prepared/i019_mangosteen_grid.jpg
✅ Saved ./data/prepared/i020_mangosteen_grid.jpg
✅ Saved ./data/prepa

### convert to 2 x 1 grid (224 * 112)

In [1]:
import os
from PIL import Image

input_dir = "./data/raw/full/output_normalized"
output_dir = "./data/prepared-224-112"
os.makedirs(output_dir, exist_ok=True)

TARGET_W, TARGET_H = 224, 224
CELL_W, CELL_H = TARGET_W // 2, TARGET_H  # 112 × 224

def fit_into_cell(img: Image.Image, cell_w: int, cell_h: int, bg=(255, 255, 255)) -> Image.Image:
    """
    ย่อภาพให้พอดีในกรอบ cell โดยรักษาอัตราส่วน แล้วแพดพื้นหลังให้เต็ม cell_w × cell_h
    """
    img = img.convert("RGB")
    # คำนวณสเกลเพื่อไม่ให้เกินขนาด cell (รักษาอัตราส่วน)
    scale = min(cell_w / img.width, cell_h / img.height)
    new_w = max(1, int(img.width * scale))
    new_h = max(1, int(img.height * scale))
    img_resized = img.resize((new_w, new_h), Image.Resampling.LANCZOS)

    canvas = Image.new("RGB", (cell_w, cell_h), bg)
    x = (cell_w - new_w) // 2
    y = (cell_h - new_h) // 2
    canvas.paste(img_resized, (x, y))
    return canvas

# --- รวบไฟล์และจัดกลุ่มตาม prefix (i001, i002, ...) ---
files = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(".jpg") or f.lower().endswith(".png")])

groups = {}
for f in files:
    # คาดว่าไฟล์ชื่อรูปแบบ: i001_upper.jpg, i001_side.jpg (ยืดหยุ่นต่อ _mangosteen_1 ก็ได้)
    base = f.split(".")[0]
    idx = base.split("_")[0]  # เช่น i001
    key_lower = f.lower()
    if idx not in groups:
        groups[idx] = {}
    if "upper" in key_lower:
        groups[idx]["upper"] = f
    elif "side" in key_lower:
        groups[idx]["side"] = f

# --- รวมภาพ 2×1: [left=upper] | [right=side] ---
for idx, imgs in sorted(groups.items()):
    if not all(k in imgs for k in ["upper", "side"]):
        print(f"⚠️ Skip {idx}, missing upper/side")
        continue

    upper = Image.open(os.path.join(input_dir, imgs["upper"]))
    side  = Image.open(os.path.join(input_dir, imgs["side"]))

    # ทำให้พอดี cell 112×224 โดยไม่บิดสัดส่วน
    cell_left  = fit_into_cell(upper, CELL_W, CELL_H, bg=(255, 255, 255))
    cell_right = fit_into_cell(side,  CELL_W, CELL_H, bg=(255, 255, 255))

    # สร้างภาพผลลัพธ์ 224×224 พื้นขาว
    out = Image.new("RGB", (TARGET_W, TARGET_H), (255, 255, 255))
    out.paste(cell_left,  (0, 0))
    out.paste(cell_right, (CELL_W, 0))

    out_path = os.path.join(output_dir, f"{idx}_mangosteen_2x1_224.jpg")
    out.save(out_path, quality=95)
    print(f"✅ Saved {out_path}")


✅ Saved ./data/prepared-224-112/i001_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i002_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i003_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i004_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i005_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i006_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i007_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i008_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i009_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i010_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i011_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i012_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i013_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i014_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i015_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i016_mangosteen_2x1_224.jpg
✅ Saved ./data/prepared-224-112/i017_man